## SeisNoise Implementation for OOI Cabled Array Data

### Standard SeisNoise code adapted to save cross correlation files of multiple years between two specific stations


In [1]:
using SeisNoise, SeisIO, Plots, HDF5
fs = 40. # sampling frequency in Hz
#freqmin,freqmax = 0.1,0.2 # min and max frequencies
freqmin,freqmax = 0.2,1 
cc_step, cc_len = 450, 1800 # corrleation step and length in S
maxlag = 60. # maximum lag time in correlation

for j in 2015:2020
    if j==2020
        m=1
        k=365
    elseif j==2015
        m=63
        k=360
    elseif j==2016
        m=1
        k=366
    else
        m=1
        k=365
    end
        
        
    for i in m:k 

        if i<10
            a=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/00",string(i),"/HYS13.OO.",string(j),".00",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/00",string(i),"/HYS13.OO.",string(j),".00",string(i),".SHN")
        elseif i>=10 && i<100
            a=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/0",string(i),"/HYS13.OO.",string(j),".0",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/0",string(i),"/HYS13.OO.",string(j),".0",string(i),".SHN")
        elseif i>=100 && i<1000
            a=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/",string(i),"/HYS13.OO.",string(j),".",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS13/",string(j),"/",string(i),"/HYS13.OO.",string(j),".",string(i),".SHN")
        end

        y=0
        try S1=read_data(a) catch; y=1 end
        try S2=read_data(b) catch; y=1 end
        try process_raw!(S1,fs) catch; y=1 end
        try process_raw!(S2,fs) catch; y=1 end
        
        if y==0
            S1=read_data(a)
            S2=read_data(b)
            process_raw!(S1,fs)
            process_raw!(S2,fs)
            
            if length(S1.x[])==3456000 && length(S2.x[])==3456000
                R = RawData.([S1,S2],cc_len,cc_step)
                detrend!.(R)
                taper!.(R)
                bandpass!.(R,freqmin,freqmax,zerophase=true)
                FFT = rfft.(R)
                whiten!.(FFT,freqmin,freqmax)
                C = correlate(FFT[1],FFT[2],maxlag)
                save_corr(C,"/data/wsd01/corrdata/hfreq/")
            end
            
        end
    
    end
end


┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/home/seismic/.julia/packages/Plots/1KWPG/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007fe27a9b847f, Ptr{Nothing} @0x00007fe27aa5770c, Ptr{Nothing} @0x00007fe27aa57e75, Ptr{Nothing} @0x00007fe27aa5790d, Ptr{Nothing} @0x00007fe27aa586d1, Ptr{Nothing} @0x00007fe27aa592f7, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x00007fe27aa767a3, Ptr{Nothing} @0x00007fe27aa76e3c, Ptr{Nothing} @0x00007fe27aa769e4, Ptr{Nothing} @0x00007fe27aa769e4, Ptr{Nothing} @0x00007fe27aa78629, Ptr{Nothing} @0x00007fe25a5ab83a, Ptr{Nothing} @0x00007fe27aa38e09, Ptr{Nothing} @0x00007fe25a69b35a, Ptr{Nothing} @0x00007fe25a1b8c15, Ptr{Nothing} @0x00007fe27aa38e09, Ptr{Nothing} @0x00007fe27aa495d6, Ptr{Nothing} @0x00007fe2580dcfaa, Ptr{Nothing} @0x00007fe27aa38e09, Ptr{Nothing} @0x00007fe27aa57e95, Ptr{Nothing} @0x00007fe2

### standard seisnoise code for plotting cross correlation function for one day between two stations

In [ ]:
using SeisNoise, SeisIO, Plots
fs = 40. # sampling frequency in Hz
freqmin,freqmax = 0.1,0.2 # min and max frequencies
cc_step, cc_len = 450, 1800 # corrleation step and length in S
maxlag = 60. # maximum lag time in correlation

j=2020
i=205
        if i<10
            a=string("/data/wsd01/OOI-Data/HYSB1/",string(j),"/00",string(i),"/HYSB1.OO.",string(j),".00",string(i),".SHZ")
            b=string("/data/wsd01/OOI-Data/HYS14/",string(j),"/00",string(i),"/HYS14.OO.",string(j),".00",string(i),".SHZ")
        elseif i>=10 && i<100
            a=string("/data/wsd01/OOI-Data/HYSB1/",string(j),"/0",string(i),"/HYSB1.OO.",string(j),".0",string(i),".SHZ")
            b=string("/data/wsd01/OOI-Data/HYS14/",string(j),"/0",string(i),"/HYS14.OO.",string(j),".0",string(i),".SHZ")
        elseif i>=100 && i<1000
            a=string("/data/wsd01/OOI-Data/HYSB1/",string(j),"/",string(i),"/HYSB1.OO.",string(j),".",string(i),".SHZ")
            b=string("/data/wsd01/OOI-Data/HYS14/",string(j),"/",string(i),"/HYS14.OO.",string(j),".",string(i),".SHZ")
        end

        S1=read_data(a)
        S2=read_data(b)

        process_raw!(S1,fs)
        process_raw!(S2,fs)

        R = RawData.([S1,S2],cc_len,cc_step)
        detrend!.(R)
        taper!.(R)
        bandpass!.(R,freqmin,freqmax,zerophase=true)
        FFT = rfft.(R)
        whiten!.(FFT,freqmin,freqmax)
        C = correlate(FFT[1],FFT[2],maxlag)
        clean_up!(C,freqmin,freqmax)
        abs_max!(C)
        plot(C)


### Save correlation matrix for one day -> debugging portal

In [ ]:
j=2017
i=287        
        if i<10
            a=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/00",string(i),"/HYS12.OO.",string(j),".00",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/00",string(i),"/HYS12.OO.",string(j),".00",string(i),".SHN")
        elseif i>=10 && i<100
            a=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/0",string(i),"/HYS12.OO.",string(j),".0",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/0",string(i),"/HYS12.OO.",string(j),".0",string(i),".SHN")
        elseif i>=100 && i<1000
            a=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/",string(i),"/HYS12.OO.",string(j),".",string(i),".SHE")
            b=string("/data/wsd01/OOI-Data/HYS12/",string(j),"/",string(i),"/HYS12.OO.",string(j),".",string(i),".SHN")
        end

        y=0
        try S1=read_data(a) catch; y=1 end
        try S2=read_data(b) catch; y=1 end
        try process_raw!(S1,fs) catch; y=1 end
        try process_raw!(S2,fs) catch; y=1 end
        
        if y==0
            S1=read_data(a)
            S2=read_data(b)
            process_raw!(S1,fs)
            process_raw!(S2,fs)

            R = RawData.([S1,S2],cc_len,cc_step)
            detrend!.(R)
            taper!.(R)
            bandpass!.(R,freqmin,freqmax,zerophase=true)
            FFT = rfft.(R)
            whiten!.(FFT,freqmin,freqmax)
            C = correlate(FFT[1],FFT[2],maxlag)
            save_corr(C,"/data/wsd01/corrdata/")
        end

### checks the available date entries withing the JLD file

In [ ]:
using JLD2
jldopen("/data/wsd01/corrdata/hfreq/OO.HYS14..BHZ.OO.HYSB1..BHZ.jld2")

# /data/wsd01/corrdata/OO.HYS11..SHE.OO.HYS11..SHN.jld2
# /data/wsd01/corrdata/OO.HYS12..SHE.OO.HYS12..SHN.jld2
# /data/wsd01/corrdata/OO.HYS13..SHE.OO.HYS13..SHN.jld2
# /data/wsd01/corrdata/OO.HYS14..BHE.OO.HYS14..BHN.jld2
# /data/wsd01/corrdata/OO.HYSB1..BHE.OO.HYSB1..BHN.jld2
# /data/wsd01/corrdata/OO.HYS14..BHZ.OO.HYSB1..BHZ.jld2

### Loads correlation for one day --> EN: between East and North , ZZ: between Z and Z channel

In [ ]:
d=load_corr("/data/wsd01/corrdata/OO.HYS14..BHE.OO.HYS14..BHN.jld2","EN",day="2016-01-05")

### loads Saved Correlation functions and Stack them daily ---> Plot and Save figure of stacked version

In [4]:
d=load_corr("/data/wsd01/corrdata/hfreq/OO.HYS14..BHZ.OO.HYSB1..BHZ.jld2","ZZ")
z6=stack(d)

CorrData with 1252 Corrs
      NAME: "OO.HYS14..BHZ.OO.HYSB1..BHZ"      
        ID: "2015-03-09"                       
       LOC: 0.0 N, 0.0 E, 0.0 m
      COMP: "ZZ"                               
   ROTATED: false                              
 CORR_TYPE: "CC"                               
        FS: 40.0
      GAIN: 1.0
   FREQMIN: 1.0
   FREQMAX: 5.0
    CC_LEN: 1800.0
   CC_STEP: 450.0
  WHITENED: true                               
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 7 entries                          
      DIST: 0.0
       AZI: 0.0
       BAZ: 0.0
    MAXLAG: 60.0
         T: 2015-03-09T00:00:00                …
      CORR: 4801×1252 Matrix{Float32}          


In [5]:
clean_up!(z6,freqmin,freqmax)
abs_max!(z6)
plot(z6)
plot!(size=(1200,1200))
savefig("HYS14HYSB1_hfreq.pdf")

In [ ]:
pwd()
ls()

### terminal commands

htop
df
ls
cd


In [9]:
open compute_fft.jl

LoadError: syntax: extra token "compute_fft" after end of expression